Python
======

When I discovered Python in around the year 2000 it was a revelation.

Before that I had programmed almost entirely in C or Basic. Python
was a huge improvement in usability over those languages. (At the time I didn't know about Scheme or Lisp or Smalltalk or Javascript 
or R - all languages I strongly prefer today).Sometime in the last few years Python has experienced rapid growth as
a data science language with packages like numpy, scipy, sklearn, etc 
providing a large library base for data science. If you had to compare R and Python in terms of libraries, you'd say R 
is more the statistician's language and Python is more the data scientist's.

I personally think from a design point of view R is better in many ways. But 
some machine learning implementations are better supported in Python. You just
have to put up with it.



Jupyter
=======

Jupyter (specifically Jupyter Lab) is roughly the equivalent of Rstudio for Python. It places a much higher emphasis on Notebooks.

Docker
======

You can run a Jupyter lab session from inside an extended rocker/verse image with these lines:

```
RUN apt update -y && apt install -y python3-pip
RUN pip3 install jupyter jupyterlab
```

A similar command line to the one we've been using to start RStudio can start Jupyter:

```
docker run -p 8765:8765 -v \
 `pwd`:/home/rstudio \
  -e PASSWORD=some_password \
  -it l14 sudo -H -u rstudio /bin/bash \
  -c "cd ~/; jupyter lab --ip 0.0.0.0 --port 8765"
```

Note that you want to replace `pwd` with something like "$(pwd)" if you have spaces in your path on a mac. You might need to literally type your folder location if you are running in windows.

Python Basics
=============

Python supports the usual programming language things:

In [1]:
1+1

2

Python is not, at base, an array language.

In [3]:
3 + [1,2,3]

TypeError: unsupported operand type(s) for +: 'int' and 'list'

In fact, the built in list type is polymorphic:

In [4]:
["x",1,"y",[]]

['x', 1, 'y', []]

This makes the built in lists less than efficient for doing numerical computations. We'll have to use a library to implement similar features in Python.

Object Orientation
==================

Python is object oriented in a much more traditional sense than R. Everything in Python is an object. Unlike in R, the primary way to experience their objectness is by calling methods:

In [7]:
l = [1,2,3]
l.append("Some value")
l

[1, 2, 3, 'Some value']

Note a few things here.

1. `=` is the assignment operator. The only one. Unlike R.
2. we use `.` to mean "access a method or property of the object before the ." In R "." is just another character that might appear in a variable name and has no special properties. R's `$` is the closest thing to `.` but `.` does more. `$` is not an allowed character in python variables and doesn't have a meaning.
3. `l.append` is the name of the "append" method on the "l" object (a list). Note that calling append "mutates" the list bound to "l". This is atypical for R where we typically create new values rather than mutate old ones.
4. Note again that we can put different types of things in our list.

Numbers are, of course, immutable. 

Everything really is an object in a sense. You can call methods on numbers:

In [13]:
(10).to_bytes(8,"little")


b'\n\x00\x00\x00\x00\x00\x00\x00'

Variables, Bindings, Environments, Functions
============================================

Python is somewhat simple compared to R here. `=` introduces a variable bindings in the local scope exclusively. At the top level `=` introduces a global variable.
Here we create a binding to "x" of "10" at the top level. The "=" sign creates a local binding inside the body of the function f. In the body of the function "x" refers to that binding.

In [17]:
x = 10
def f():
    x=11
    return x
[f(),x]

[11, 10]

Things to note:
    
1. Python is whitespace sensitive. The body of functions must be indented compared to the enclosing context. That ":" at the end of the `def` line is also required.
2. Unlike in R we _must_ explicitely return a value from functions using "return". "return" terminates the function immediately if it is placed in some non-tail position.
3. These are some of the worst features of python that tell you it was designed by a rube.

Mutating an Enclosing Variable
==============================

If you want to change global variable binding (as you would do with "<<-" in R) you have to make this desire known by declaring the variable global in your function.

In [20]:
y = 10;
def set_y(v):
    global y
    y = v
    return y
[set_y(100),y]

[100, 100]

Things become increasingly absurd as you may nest scopes:

In [21]:
def absurdity():
    z = 10;
    def inner_absurdity(v):
        nonlocal z   
        z = v
    inner_absurdity("what");
    return z;
absurdity()

'what'

A global variable cannot be declared "nonlocal" even though the relationship which obtains between a function scope and a global scope is the same as one obtained between two function scopes. This doesn't really matter that much but it chaps my britches.

Conditionals and Loops
======================

If
--

In [22]:
x = 1
y = 2
if x < y :
    print("Hiho")
else:
    print("Silver")

Hiho


Note the ":" and indentation. Also note that you do not need an enclosing () for the conditions.
If statements can have many legs:

In [ ]:
if x < y:
    print("smaller")
elif x == y:
    print("equal")
else:
    print("larger")

Finally note that if statements don't produce any values. They only perform side effects. The following function returns no value at all.

In [23]:
def if_example(x,y):
    if x < y:
        "smaller"
    elif x == y:
        "equal"
    else:
        "larger"

It should look like this:

In [25]:
def if_example(x,y):
    if x < y:
        return "smaller"
    elif x == y:
        return "equal"
    else:
        return "larger"

Loops and Comprehensions
------------------------

Loops come in a few flavors.

In [26]:
for x in [1,2,3]:
    print(x)

1
2
3


In [27]:
for x in range(10):
    print(x)

0
1
2
3
4
5
6
7
8
9


This is a good time to remark upon the fact that python is zero indexed based. Thus the `range` function returns a list of indexes for an arrange of the input length.

While loops are predictable at this point:

In [28]:
x = 0;
while x < 10:
    print(x)
    x = x + 1;

0
1
2
3
4
5
6
7
8
9


We can see from this example that for and while loops do not create their own contexts in their body. If they did we'd need a "global x" above.

Comprehensions
--------------

Comprehensions are a nice feature if you don't know about functional programming. They let you construct new lists from old lists and often this is what you want when you think you want a loop:

In [30]:
x = [1,2,3]
x_plus_one = [e + 1 for e in x]
x_plus_one

[2, 3, 4]

Comprehensions can get somewhat complex: 

In [32]:
def odd(n):
    return (n % 2) == 1

[e + 1 for e in range(10) if odd(e)]

[2, 4, 6, 8, 10]

Anonymous Functions
===================

Another way that Python is broken is that anonymous functions are pretty limited. Note that we always have to give a name during a `def` in Python. In R, the `function` form returns a function which we bind to a name via `<-`. We don't have to give it a name and often we don't. 

In python there is no equivalent. There are `lambda` expressions, however.

In [34]:
def map(l, f):
    return [f(x) for x in l]
    
map(range(10),lambda x: x*2)

[0, 2, 4, 6, 8, 10, 12, 14, 16, 18]

Lambda expressions are limited to a single expression in their body. You cannot create new variable bindings inside of them.  This is a big limitation on their expressiveness.

All is not lost, however. Functions in Python are first order objects, so you can say:

In [35]:
def times_3(x):
    return x * 3
map(range(10), times_3)

[0, 3, 6, 9, 12, 15, 18, 21, 24, 27]

Since we can nest function definitions this gives us most of what we want. Note that `lambda` expressions are the only sorts of functions where we don't need to say "return" to return a value.